In [2]:
import pandas as pd 
import numpy as np 
import tqdm
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [27]:

URI = "neo4j+s://d4e7c69a.databases.neo4j.io"
AUTH = ("neo4j", "8gGwVhSx2-ycIPiPGOWejHAhufieq2XOOrkOAizxa1E")
driver = GraphDatabase.driver(URI, auth=AUTH)

In [41]:
co2_mat = pd.read_excel("co2mat.xlsx", sheet_name="Sheet2")
mat = co2_mat["Eng"].to_list()

In [43]:
co2_mat

,Total,Fabrication,Élimination,Contenu dans le produit,Produit,Eng
0,861.0,801.0,60.500,0.00,"Blindage de fouille, paroi parisienne, étayé","Excavation shielding, Parisian wall, supported"
1,821.0,761.0,60.500,0.00,"Blindage de fouille, paroi parisienne, flottant","Excavation shielding, Parisian wall, floating"
2,535.0,495.0,40.000,0.00,"Blindage de fouille, paroi parisienne, fixe","Excavation shielding, Parisian wall, fixed"
3,121.0,112.0,8.780,0.00,"Blindage de fouille, paroi clouée","Excavation shielding, nailed wall"
4,270.0,256.0,14.200,0.00,"Blindage de fouille, paroi berlinoise, en port...","Excavation shielding, Berlin wall, cantilevered"
...,...,...,...,...,...,...
294,470.0,418.0,51.300,160.00,"Cuisine, bois massif, 16 éléments","Kitchen, solid wood, 16 elements"
295,2770.0,2720.0,51.000,3.45,"Cuisine, métal, 16 éléments","Kitchen, metal, 16 elements"
296,632.0,557.0,74.800,223.00,"Cuisine, bois reconstitué, 16 éléments","Kitchen, reconstituted wood, 16 elements"
297,48.6,43.6,4.920,0.00,Armoire-évier en acier chromé,Chrome steel sink cabinet


In [49]:
import pandas as pd 
import pandas as pd
import numpy as np
from neo4j import GraphDatabase
import streamlit as st
import ast

co2_mat = pd.read_excel("co2mat.xlsx", sheet_name="Sheet2")
mat = co2_mat["Eng"].to_list()



class Component:
    def __init__(self):
        self.carbon = 0
        self.social = 0
        self.rvi = 0

    def calculate_social(self):
        self.social = 1

    def calculate_carbon(self):
        self.carbon = 1

    def calculate_rvi(self):
        self.calculate_social()
        self.calculate_carbon()
        self.rvi = self.carbon * 2 + self.social * 5
        return self.rvi

def fetch_component_from_neo4j(driver, component_name):
    with driver.session() as session:
        query = "MATCH (n:Component) WHERE n.name = $name RETURN n.name"
        result = session.run(query, name=component_name)
        components = [record["n.attributes"] for record in result]
        print(components)

        return components

def fetch_component_attributes_from_neo4j(driver, component_name):
    with driver.session() as session:
        query = """
        MATCH (n:Component {name: $name})
        RETURN n.attributes
        """
        results = session.run(query, name=component_name)
        components = [record.get("n.attributes", {}) for record in results]
        return components


def display_rvi_streamlit(driver):
    #with st.form(key='component_form'):
    #    search_name = st.text_input('Enter Component Name:')
    #    submit_button = st.form_submit_button(label='Search')

    #if submit_button:
    #components_names = fetch_component_from_neo4j(driver, "door")
    components_attributes = fetch_component_attributes_from_neo4j(driver, "door")

    if components_attributes:
        component_rvi_list = []

        for component_att in components_attributes:
            comp = Component()
            material = ast.literal_eval(component_att)["Material"]
                # Search for rows in the DataFrame where 'Eng' matches the material name
            matching_rows = co2_mat[co2_mat['Eng'] == material]

            # If there are matching rows, extract the required data
            if not matching_rows.empty:
                for _, row in matching_rows.iterrows():
                    total = row['Total']
                    fabrication = row['Fabrication']
                    elimination = row['Élimination']
                    contenu_dans_le_produit = row['Contenu dans le produit']
                    # Process the data as needed, e.g., print or store in a dictionary
                    print(f"Material: {material}, Total: {total}, Fabrication: {fabrication}, Elimination: {elimination}, Contained: {contenu_dans_le_produit}")

        for component, rvi in component_rvi_list:
            
            print(type(ast.literal_eval(component)))
    else: print("ok")


In [50]:
display_rvi_streamlit(driver)

Material: Glass wool, Total: 1.1, Fabrication: 1.04, Élimination: 0.0592, Contenu: 0.0
Material: Exterior wooden doors, with glazing, Total: 85.7, Fabrication: 76.0, Élimination: 9.7, Contenu: 13.0
Material: Interior door, bedroom door, wood, glazing, wooden frame, Total: 59.7, Fabrication: 52.8, Élimination: 6.92, Contenu: 11.0
Material: Exterior wooden doors, with glazing, Total: 85.7, Fabrication: 76.0, Élimination: 9.7, Contenu: 13.0
Material: Interior door, bedroom door, wood, glazing, wooden frame, Total: 59.7, Fabrication: 52.8, Élimination: 6.92, Contenu: 11.0
Material: Glass wool, Total: 1.1, Fabrication: 1.04, Élimination: 0.0592, Contenu: 0.0


In [4]:

IFC = pd.read_csv("IFC_processed.csv") 
eBKP = pd.read_csv("eBKP_processed1.csv") 
MF = pd.read_csv("MF_processed.csv") 
UNI = pd.read_excel("Uniclass2015_Pr.xlsx", header=2) 

def read_and_encode(model, df):
    sentences = df.tolist()
    embeddings = model.encode(sentences)

    return np.array(embeddings)


def save_embeddings(embeddings, file_name):
    np.save(file_name, embeddings)

# Process and save embeddings for each file
embeddings_ifc = read_and_encode(model, IFC.IFC)
save_embeddings(embeddings_ifc, 'embeddings_ifc.npy')

embeddings_ebkp = read_and_encode(model, eBKP['Element designation_EN'])
save_embeddings(embeddings_ebkp, 'embeddings_ebkp.npy')

embeddings_mf = read_and_encode(model, MF['label'])
save_embeddings(embeddings_mf, 'embeddings_mf.npy')

embeddings_uni = read_and_encode(model, UNI['Title'])
save_embeddings(embeddings_uni, 'embeddings_uni.npy')


In [2]:
import pandas as pd 
import numpy as np 
import tqdm
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


IFC = pd.read_csv("IFC_processed.csv") 
eBKP = pd.read_csv("eBKP_processed1.csv") 
MF = pd.read_csv("MF_processed.csv") 
UNI = pd.read_excel("Uniclass2015_Pr.xlsx", header=2) 

def read_and_encode(model, df):
    sentences = df.tolist()
    embeddings = model.encode(sentences)

    return np.array(embeddings)


def save_embeddings(embeddings, file_name):
    np.save(file_name, embeddings)

# Process and save embeddings for each file
embeddings_ifc = read_and_encode(model, IFC.IFC)
save_embeddings(embeddings_ifc, 'embeddings_ifc_en.npy')

embeddings_ebkp = read_and_encode(model, eBKP['Element designation_EN'])
save_embeddings(embeddings_ebkp, 'embeddings_ebkp_en.npy')

embeddings_mf = read_and_encode(model, MF['label'])
save_embeddings(embeddings_mf, 'embeddings_mf_en.npy')

embeddings_uni = read_and_encode(model, UNI['Title'])
save_embeddings(embeddings_uni, 'embeddings_uni_en.npy')


In [ ]:
def read_and_encode(model, df):

    sentences = df.tolist()

    # Initialize tqdm for progress tracking
    embeddings = []
    for sentence in tqdm(sentences):
        embedding = model.encode(sentence, convert_to_numpy=True)
        embeddings.append(embedding)

    return np.array(embeddings)

def save_embeddings(embeddings, file_name):
    # Save the embeddings to a file
    np.save(file_name, embeddings)

# Process and save embeddings for each file
embeddings_ifc = read_and_encode('ifc.csv')
save_embeddings(embeddings_ifc, 'embeddings_ifc.npy')

embeddings_ebkp = read_and_encode('ebkp.csv')
save_embeddings(embeddings_ebkp, 'embeddings_ebkp.npy')

embeddings_mf = read_and_encode('mf.csv')
save_embeddings(embeddings_mf, 'embeddings_mf.npy')

embeddings_uni = read_and_encode('uni.csv')
save_embeddings(embeddings_uni, 'embeddings_uni.npy')

In [5]:
from neo4j import GraphDatabase

URI = "neo4j+ssc://d4e7c69a.databases.neo4j.io"
AUTH = ("neo4j", "8gGwVhSx2-ycIPiPGOWejHAhufieq2XOOrkOAizxa1E")
driver = GraphDatabase.driver(URI, auth=AUTH)

def create_cypher_queries_from_json(json_data):
    queries = []

    # Iterate through the items in the JSON object
    for item_key, item_value in json_data.items():
        if not item_value:
            continue

        # Replace problematic characters in keys for node labels
        node_label = item_key.replace("Ifc", "Ifc_").replace('.', '_')
        parent_label = item_value.get("Parent", "").replace("Ifc", "Ifc_").replace('.', '_')

        # Start building the query for the component node
        component_query = f"MERGE (component:{node_label} {{name: $component_name"

        # Initialize a dictionary for parameters
        params = {'component_name': node_label}

        # Add properties to the component node from the JSON
        for pset_name, pset_details in item_value.get("Psets", {}).items():
            for prop_name, prop_details in pset_details.get("Properties", {}).items():
                # Add property to the query
                safe_prop_name = prop_name.replace(' ', '_')  
                component_query += f", {safe_prop_name}: ${safe_prop_name}"
                params[safe_prop_name] = " "

        # Close the curly braces for the MERGE statement
        component_query += "})"

        # If a parent label exists, add relationship queries
        if parent_label:
            parent_query = (
                f" MERGE (parent:{parent_label} {{name: $parent_name}})"
                " MERGE (component)-[:IS_A]->(parent)"
            )
            params['parent_name'] = parent_label
            component_query += parent_query

        # Add the completed query and parameters to the list
        queries.append({'query': component_query, 'params': params})

    return queries

import json
import pandas as pd

# Assuming you have a JSON file named 'data.json'
with open('IFC.json', 'r') as file:
    data = json.load(file)

queries= create_cypher_queries_from_json(data["Domain"]["Classifications"])

#run_cypher_query(driver, " ".join(queries))
def run_cypher_query(driver, query, params):
    with driver.session() as session:
        session.run(query, **params)

for query_info in queries[253:]:
    run_cypher_query(driver, query_info['query'], query_info['params'])


Failed to read from defunct connection IPv4Address(('d4e7c69a.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('35.205.213.74', 7687)))


SessionExpired: Failed to read from defunct connection IPv4Address(('d4e7c69a.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('35.205.213.74', 7687)))

In [7]:
from neo4j import GraphDatabase

URI = "neo4j+ssc://d4e7c69a.databases.neo4j.io"
AUTH = ("neo4j", "8gGwVhSx2-ycIPiPGOWejHAhufieq2XOOrkOAizxa1E")
driver = GraphDatabase.driver(URI, auth=AUTH)

def create_cypher_queries_from_json(json_data):
    queries = []

    # Iterate through the items in the JSON object
    for item_key, item_value in json_data.items():
        if not item_value:
            continue

        # Replace problematic characters in keys for node labels
        node_label = item_key.replace("Ifc", "Ifc_").replace('.', '_')
        parent_label = item_value.get("Parent", "").replace("Ifc", "Ifc_").replace('.', '_')

        # Start building the query for the component node
        component_query = f"MERGE (component:{node_label} {{name: $component_name"

        # Initialize a dictionary for parameters
        params = {'component_name': node_label}

        # Add properties to the component node from the JSON
        for pset_name, pset_details in item_value.get("Psets", {}).items():
            for prop_name, prop_details in pset_details.get("Properties", {}).items():
                # Add property to the query
                safe_prop_name = prop_name.replace(' ', '_')  
                component_query += f", {safe_prop_name}: ${safe_prop_name}"
                params[safe_prop_name] = " "

        # Close the curly braces for the MERGE statement
        component_query += "})"

        # If a parent label exists, add relationship queries
        if parent_label:
            parent_query = (
                f" MERGE (parent:{parent_label} {{name: $parent_name}})"
                " MERGE (component)-[:IS_A]->(parent)"
            )
            params['parent_name'] = parent_label
            component_query += parent_query

        # Add the completed query and parameters to the list
        queries.append({'query': component_query, 'params': params})

    return queries

import json
import pandas as pd

# Assuming you have a JSON file named 'data.json'
with open('IFC.json', 'r') as file:
    data = json.load(file)

queries= create_cypher_queries_from_json(data["Domain"]["Classifications"])

#run_cypher_query(driver, " ".join(queries))
def run_cypher_query(driver, query, params):
    with driver.session() as session:
        session.run(query, **params)

for query_info in queries:
    run_cypher_query(driver, query_info['query'], query_info['params'])


Failed to write data to connection ResolvedIPv4Address(('35.205.213.74', 7687)) (ResolvedIPv4Address(('35.205.213.74', 7687)))


In [1]:
import json
import pandas as pd
# Assuming you have a JSON file named 'data.json'
with open('IFC.json', 'r') as file:
    data = json.load(file)

item_key_list = []
pset_name_list=[]
prop_name_list=[]
prop_details_list=[]
for item_key, item_value in data["Domain"]["Classifications"].items():
    # Add properties and property sets
    for pset_name, pset_details in item_value.get("Psets", {}).items():
        for prop_name, prop_details in pset_details.get("Properties", {}).items():
            if prop_details["type"] == "string" or prop_details["type"] == "real":
                prop_details = prop_details["type"]
            elif "values" in prop_details.keys():
                prop_details = str(prop_details["values"])
            else:
                prop_details = str(prop_details)

            item_key_list.append(item_key)
            prop_name_list.append(prop_name)
            prop_details_list.append(prop_details)

In [15]:
def get_attributes(classification):
    prop_name_list=[]
    for pset_name, pset_details in data["Domain"]["Classifications"][classification].get("Psets", {}).items():
        for prop_name, prop_details in pset_details.get("Properties", {}).items():
            if prop_details["type"] == "string" or prop_details["type"] == "real":
                prop_details = prop_details["type"]
            elif "values" in prop_details.keys():
                prop_details = str(prop_details["values"])
            else:
                prop_details = str(prop_details)

            prop_name_list.append((prop_name, prop_details))
    return prop_name_list

In [16]:
prop_name_list 

[('ProcessCapacity',
  "{'type': 'integer', 'Description': 'The number of units that can be processed in the time defined in ProcessPerformance'}"),
 ('ProcessItem',
  "['BARREL', 'CGT', 'NOTKNOWN', 'OTHER', 'PASSENGER', 'TEU', 'TONNE', 'UNSET', 'VEHICLE']"),
 ('ProcessPerformance', 'string'),
 ('AcousticRating', 'string'),
 ('DurabilityRating', 'string'),
 ('FireExit', "['TRUE', 'FALSE']"),
 ('FireRating', 'string'),
 ('GlazingAreaFraction', 'real'),
 ('HandicapAccessible', "['TRUE', 'FALSE']"),
 ('HasDrive', "['TRUE', 'FALSE']"),
 ('HygrothermalRating', 'string'),
 ('Infiltration', 'real'),
 ('IsExternal', "['TRUE', 'FALSE']"),
 ('MechanicalLoadRating', 'string'),
 ('Reference', 'string'),
 ('SecurityRating', 'string'),
 ('SelfClosing', "['TRUE', 'FALSE']"),
 ('SmokeStop', "['TRUE', 'FALSE']"),
 ('Status',
  "['DEMOLISH', 'EXISTING', 'NEW', 'NOTKNOWN', 'OTHER', 'TEMPORARY', 'UNSET']"),
 ('ThermalTransmittance', 'real'),
 ('WaterTightnessRating', 'string'),
 ('WindLoadRating', 'string

In [10]:
table = pd.DataFrame({
    "item_key": item_key_list,
    "prop_name": prop_name_list,
    "prop_details": prop_details_list
})

In [14]:
import pandas as pd 
from neo4j import GraphDatabase

URI = "neo4j+ssc://d4e7c69a.databases.neo4j.io"
AUTH = ("neo4j", "8gGwVhSx2-ycIPiPGOWejHAhufieq2XOOrkOAizxa1E")
driver = GraphDatabase.driver(URI, auth=AUTH)

def create_cypher_queries_mf(mf):
    queries = []
    for index, row in mf.iterrows():
        # Clean up the code to be a valid Cypher label by replacing spaces with underscores and prefixing with a letter
        component_label = 'C' + row['code'].replace(' ', '_').replace('.', '_')
        parent_label = 'C' + row['parent'].replace(' ', '_').replace('.', '_') if row['parent'] != None else False 
        c_label = 'C' + row['label']

        # Create the MERGE queries
        component_query = f"MERGE (component:{component_label} {{name: $component_label, label:$c_label}})"
        params = {'component_label': component_label, 'c_label': c_label}

        if parent_label:
            parent_query = (
                f" MERGE (parent:{parent_label} {{name: $parent_label}})"
                " MERGE (component)-[:IS_A]->(parent)"
            )
            params['parent_label'] = parent_label
        
            component_query += parent_query
        
        # Parameters for the query
        
        # Add the query and params to the list
        queries.append({'query': component_query, 'params': params})
        
    return queries

mf = pd.read_csv("MF_processed.csv")
def find_parent(row, df):
    for i in range(row.name - 1, -1, -1):
        if df.at[i, 'level'] == row['level'] - 1:
            return df.at[i, 'code']
    return None

mf['parent'] = mf.apply(lambda row: find_parent(row, mf), axis=1)

queries= create_cypher_queries_mf(mf)

def run_cypher_query(driver, query, params):
    with driver.session() as session:
        session.run(query, **params)

for query_info in queries:
    run_cypher_query(driver, query_info['query'], query_info['params'])

In [2]:

from neo4j import GraphDatabase
import pandas as pd 

URI = "neo4j+ssc://d4e7c69a.databases.neo4j.io"
AUTH = ("neo4j", "8gGwVhSx2-ycIPiPGOWejHAhufieq2XOOrkOAizxa1E")
driver = GraphDatabase.driver(URI, auth=AUTH)

def create_cypher_queries_eBKP(df):
    queries = []
    for index, row in df.iterrows():
        # Clean up the code to be a valid Cypher label by replacing spaces with underscores and prefixing with a letter
        component_label = 'B_' + row['code'].replace(' ', '_').replace('.', '_')
        c_label = 'B_' + row['Translated_Text']

        # Create the MERGE queries
        component_query = f"MERGE (component:{component_label} {{name: $component_label, label:$c_label}})"
        params = {'component_label': component_label, 'c_label': c_label}
        
        
        # Parameters for the query
        
        # Add the query and params to the list
        queries.append({'query': component_query, 'params': params})
        
    return queries

eBKP = pd.read_csv("eBKP_processed.csv")

queries= create_cypher_queries_eBKP(eBKP)

def run_cypher_query(driver, query, params):
    with driver.session() as session:
        session.run(query, **params)

for query_info in queries:
    run_cypher_query(driver, query_info['query'], query_info['params'])

In [15]:
queries

[{'query': 'MERGE (component:C00_00_00 {name: $component_label, label:$c_label})',
  'params': {'component_label': 'C00_00_00',
   'c_label': 'CProcurement and Contracting Requirements'}},
 {'query': 'MERGE (component:C00_01_01 {name: $component_label, label:$c_label})',
  'params': {'component_label': 'C00_01_01',
   'c_label': 'CProject Title Page'}},
 {'query': 'MERGE (component:C00_01_03 {name: $component_label, label:$c_label})',
  'params': {'component_label': 'C00_01_03', 'c_label': 'CProject Directory'}},
 {'query': 'MERGE (component:C00_01_05 {name: $component_label, label:$c_label})',
  'params': {'component_label': 'C00_01_05',
   'c_label': 'CCertifications Page'}},
 {'query': 'MERGE (component:C00_01_07 {name: $component_label, label:$c_label})',
  'params': {'component_label': 'C00_01_07', 'c_label': 'CSeals Page'}},
 {'query': 'MERGE (component:C00_01_10 {name: $component_label, label:$c_label})',
  'params': {'component_label': 'C00_01_10', 'c_label': 'CTable of Content